In [1]:
import pandas as pd
import scipy.io
import scipy.signal as ss
import pywt
import glob
import matplotlib.pyplot as plt
import scaleogram as scg
from scaleogram import cws
import numpy as np
from math import *
import os
import cv2
from scipy.io import loadmat


In [2]:
print(os.getcwd())
os.chdir("..")

/Users/hoangminhthao/Documents/GitHub/HeartResearch/Data Preprocessing


In [4]:
main_data_dir = os.getcwd() + "/Data set"
print(os.listdir(main_data_dir))

['.DS_Store', 'Gaussian_Filter', 'TrainingSet1', 'Median_Filter', 'Segmentation', 'README.md', 'Label.csv', 'Scaleogram_512_256', 'Output.png', 'TrainingSet3', 'TrainingSet2']


In [22]:
data_mat_files = glob.glob(main_data_dir + "/TrainingSet*/*")
data_mat_files.sort()
label_path = main_data_dir + "/Label.csv"
label_df = pd.read_csv(label_path)

In [23]:
label_df.drop(['Second_label', 'Third_label'], axis=1)
label = label_df['First_label'].tolist()

In [24]:
def data_extract(signal_raw_list, start_point):
    signal_list = []
    signal_number_list = []
    
    for signal in signal_raw_list:
        mat = scipy.io.loadmat(signal)
        signal_list.append(mat['ECG'][0][0][2][:,start_point:])
        signal_number = signal.split('/')[-1].split('.')[0]
        signal_number_list.append(signal_number)

    return (signal_list, signal_number_list)

signal_list, signal_number = data_extract(data_mat_files, 300)

In [8]:
def median_filter(signal_raw_list):
    median_fil_list = []

    for i in range(len(signal_raw_list)):
        for j in range(12):
            median_fil = ss.medfilt(signal_raw_list[i][j])
            median_fil_list.append(median_fil)    
    return median_fil_list
    
median_fil_list = median_filter(signal_list)

In [ ]:
#Segmentation
list_class_dir = ["\Segmentation\Segmentation_class_1", 
                  "\Segmentation\Segmentation_class_2", 
                  "\Segmentation\Segmentation_class_3", 
                  "\Segmentation\Segmentation_class_4", 
                  "\Segmentation\Segmentation_class_5", 
                  "\Segmentation\Segmentation_class_6", 
                  "\Segmentation\Segmentation_class_7", 
                  "\Segmentation\Segmentation_class_8", 
                  "\Segmentation\Segmentation_class_9"]

for class_dir in list_class_dir:
    os.makedirs(main_data_dir + class_dir)

# seg_class_1_path = class_1_path + "\{}_lead{}_seg{}.mat"
# seg_class_2_path = class_2_path + "\{}_lead{}_seg{}.mat"
# seg_class_3_path = class_3_path + "\{}_lead{}_seg{}.mat"
# seg_class_4_path = class_4_path + "\{}_lead{}_seg{}.mat"
# seg_class_5_path = class_5_path + "\{}_lead{}_seg{}.mat"
# seg_class_6_path = class_6_path + "\{}_lead{}_seg{}.mat"
# seg_class_7_path = class_7_path + "\{}_lead{}_seg{}.mat"
# seg_class_8_path = class_8_path + "\{}_lead{}_seg{}.mat"
# seg_class_9_path = class_9_path + "\{}_lead{}_seg{}.mat"

# class_1_dir = main_data_dir + "\Segmentation_class_1\{}_lead{}_seg{}.mat"

lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']

def segmentation(signal_filt_list, seg_len):
    count = 0
    
    for i in range(len(signal_filt_list)):
        #split
        sig_len = len(signal_filt_list[i])
        seg_num = ceil(sig_len/seg_len)
        for w in range(1,seg_num+1):
            x = floor((sig_len-seg_len)/(seg_num-1)*(w-1))
            segment = signal_filt_list[i][x:x+1600]
            seg_label = label[((i+1)/12)-1]
            save_dic_seg = {"ECG_segment": segment}
            
            class_dir = main_data_dir + "\Segmentation_class_{}".format(seg_label)
            if not os.path.exists(class_dir):
                os.mkdir(class_dir)
            seg_path = class_dir + "\{}_lead{}_seg{}.mat" 
            scipy.io.savemat(seg_path.format(signal_number[floor((i+1)/12)], lead[count], w), save_dic_seg)
            
        if count == 11:
            count = 0
        else:
            count += 1
        
            
segmentation(median_fil_list, 1600)

In [ ]:
        #scaleogram
    
fig, ax = cws(segment, 
        scales=scg.periods2scales(np.arange(1, scale)),
        figsize=(img_size[1]*px, img_size[0]*px), coi = False)
plt.axis('off')
plt.savefig(scaleogram_save_path.format(signal_filename, idx, w), 
            bbox_inches='tight', 
            # format = 'jpg', 
            pad_inches = 0)
fig.clear()
plt.close(fig)
ax.clear()
plt.cla()
plt.close("all")
        

In [ ]:
# segment_list, seg_label_df = segmentation_and_filter_scaleogram(median_fil_list, 1600, folder = 'Median_Filter')

In [ ]:
# seg_label_df.to_csv(main_data_dir + "\Segment_filter_label\Segment_Median_label.csv")

In [ ]:
# segment_file_name = "{}_lead{}_seg{}.mat"
# segment_list = []
# segment_name_list = []
# label_list = []

# def segmentation_and_filter_scaleogram1(signal_filt_list, seg_len, folder):
#     output_path = main_data_dir + "\Scaleogram_512_256\{}\{}_{}_lead{}_seg{}"
#     scg.set_default_wavelet('morl')
#     px = 1/plt.rcParams['figure.dpi']
#     lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']
#     count = 0
    
#     for i in range(len(signal_filt_list)):
#         sig_len = len(signal_filt_list[i])
#         seg_num = ceil(sig_len/seg_len)
#         for w in range(1,seg_num+1):
#             x = floor((sig_len-seg_len)/(seg_num-1)*(w-1))
#             segment = signal_filt_list[i][x:x+1600]
#             segment_list.append(segment)
#             save_dic_data = {"ECG_segment": segment}
#             label_list.append(label[floor(i/12)])
#             segment_name_list.append(segment_file_name.format(signal_number[floor(i/12)], count+1, w))
# #             scipy.io.savemat(main_data_dir + "\Segmentation\{}_lead{}_seg{}.mat".format(signal_number[floor(i/12)], count+1, w), save_dic_data)
            
#             scales = scg.periods2scales(np.arange(1, seg_len+1))
#             x_values_wvt_arr = range(0,sig_len,1)
#             for segment in segment_list:
#                 fig = cws(segment, scales=scales, figsize=(512*px, 256*px), coi = False)
# #           print(fig)
#                 plt.axis('off')
#                 plt.savefig(output_path.format(folder, folder, signal_number[floor(i/12)],count+1, w), bbox_inches='tight', pad_inches = 0)
        
#         if count == 11:
#             count = 0
#         else:
#             count += 1
#     seg_label_df = pd.DataFrame(list(zip(segment_name_list, label_list)), columns = ['Segment', 'Label'])  
    
#     return segment_list, seg_label_df

In [ ]:
# segmentation_and_filter_scaleogram1(median_fil_list[:1], 1600, folder = 'Median_Filter')

In [ ]:
# # LINH 3/1
# output_path = main_data_dir + "\Scaleogram_512_256\{}_{}_{}"
# scg.set_default_wavelet('morl')
# px = 1/plt.rcParams['figure.dpi']
# lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']

# def scaleogram(data, folder):
# #     count = 0
#     for i in range(len(data)):
#             signal_length = 1600
#             # range of scales to perform the transform
#             scales = scg.periods2scales(np.arange(1, signal_length+1))
#             x_values_wvt_arr = range(0,len(data[i]),1)

#             # plot the signal 
# #             fig1, ax1 = plt.subplots(1, 1, figsize=(9, 3.5));  
# #             ax1.plot(x_values_wvt_arr, data[i][j], linewidth=3, color='blue')
# #             ax1.set_xlim(0, signal_length)
# #             ax1.set_title("ECG")

#             # the scaleogram
#             fig = cws(data[i][:signal_length], scales=scales, figsize=(512*px, 256*px), coi = False)
# #             print(fig)
#             plt.axis('off')
#             plt.savefig(output_path.format(folder, signal_number[i],lead[j]), bbox_inches='tight', pad_inches = 0)

In [ ]:
# scaleogram(segment_list, folder = "Median_Filter")

In [ ]:
# segment_file_name = "{}_lead{}_seg{}.mat"
# # segment_list = []
# segment_name_list = []
# label_list = []

# def segmentation(signal_list, seg_len):
#     for i in range(len(signal_list)):
#         for j in range(12):
#             sig_len = len(signal_list[i][j])
#             seg_num = ceil(sig_len/seg_len)
#             for w in range(1,seg_num+1):
#                 x = floor((sig_len-seg_len)/(seg_num-1)*(w-1))
#                 segment = signal_list[i][j][x:x+1600]
#                 save_dic_data = {"ECG_segment": segment}
# #                 segment_list.append(segment)
#                 label_list.append(label[i])
#                 segment_name_list.append(segment_file_name.format(signal_number[i], j+1, w))
# #                 scipy.io.savemat(main_data_dir + "\Segmentation\{}_lead{}_seg{}.mat".format(signal_number[i], j+1, w), save_dic_data)
#     seg_label_df = pd.DataFrame(list(zip(segment_name_list, label_list)), columns = ['Segment', 'Label'])  
#     return seg_label_df

In [ ]:
# seg_label_df = segmentation(signal_list,1600)

In [ ]:
# seg_label_df.to_csv(main_data_dir + "\Segment_label.csv")

In [ ]:
# def save_data_filter(data, signal_number, folder):
# #     for i in range(len(data)):
#         save_dic_data = {"ECG": data[i]}
#         scipy.io.savemat("E:\git\HeartResearch\Data set\{}\{}.mat".format(folder,signal_number[i]), save_dic_data)

In [ ]:
# save_data_filter(median_fil_list, signal_number, folder = "Median_Filter")

In [ ]:
# output_path = main_data_dir + "\Scaleogram_512_256\{}_{}_{}"
# scg.set_default_wavelet('morl')
# px = 1/plt.rcParams['figure.dpi']
# lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']

# def scaleogram(data, folder, signal_length):
#     for i in range(len(data)):
#         for j in range(len(lead)):
            
# #             signal_length = 1600
#             # range of scales to perform the transform
#             scales = scg.periods2scales(np.arange(1, signal_length+1))
#             x_values_wvt_arr = range(0,len(data[i][j]),1)

#             # plot the signal 
#             fig1, ax1 = plt.subplots(1, 1, figsize=(9, 3.5));  
#             ax1.plot(x_values_wvt_arr, data[i][j], linewidth=3, color='blue')
#             ax1.set_xlim(0, signal_length)
#             ax1.set_title("ECG")

#             # the scaleogram
#             fig = cws(data[i][j][:signal_length], scales=scales, figsize=(512*px, 256*px), coi = False)
#             print(fig)
#             plt.axis('off')
# #             plt.savefig(output_path.format(folder, signal_number[i],lead[j]), bbox_inches='tight', pad_inches = 0)

In [ ]:
# scaleogram(median_fil_list[60:61], folder = "Median_Filter", signal_length=1650)

In [ ]:
# def gaussian_filter(signal_raw_list, std):
#     gauss_filt = []
#     n_bins = signal_raw_list.shape[2]
#     M = n_bins - 1 if n_bins % 2 == 0 else n_bins
#     window = ss.gaussian(M, std = std)

#     for signal in signal_raw_list:
#         for i in range(signal.shape[0]):
#             gauss_filter = ss.convolve(signal[i], window, mode='same', method='auto')
#             gauss_filt.append(gauss_filter)    
#     return gauss_filt

In [ ]:
gauss_filt = gaussian_filter(signal_list, std=1)

In [ ]:
gauss_filt_array = (np.array(gauss_filt)).reshape(6877,12,3000)
gauss_filt_list = []
for i in range(gauss_filt_array.shape[0]):
    gauss_filt_list.append(gauss_filt_array[i])

In [ ]:
folder = "Gaussian_Filter"
save_data_filter(gauss_filt_list, signal_number, folder)

In [ ]:
(np.array(signal_list).shape

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(signal_list[0][4])
print(signal_list[0][4])
print(type(signal_list[0][4]))

In [ ]:
plt.figure(figsize=(20,10))
y = ss.medfilt(signal_list[0][4], 17)
plt.plot(y)
print(y)